# Dags Decoupling: Création de Dags simple combiner en un seul Dag 

![atelier3](https://user-images.githubusercontent.com/123757632/231910388-d2d83283-8a29-454c-9b3e-223787b8477b.png)

# Création Dags et définition des dépendances entre les DAG dans Airflow

Lorsque deux DAG ont des relations de dépendance, il convient d'envisager de les combiner en un seul DAG, qui est généralement plus simple à comprendre. Airflow offre également une meilleure représentation visuelle de dépendances pour les tâches sur le même DAG. Cependant, il n'est parfois pas pratique de mettre tous les éléments connexes tâches sur le même DAG.

## 1. Création des deux dags Parents 'dag en amont'

Pour la création des deux dags parents , il faut commencer par la création des deux fichirs 'parent_dag_1.py' et 'parent_dag_2.py' dans le dossier 'dags' .

In [ ]:
└───dags|── parent_dag_1.py                  
        └── parent_dag_2.py

### 1 - parent_dag_1.py

In [ ]:
from airflow import models
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators import python_operator
from airflow.utils.dates import days_ago


with models.DAG(
    'dag_1',
    schedule_interval='*/1 * * * *', # Chaque minute
    start_date=days_ago(0),
    catchup=False) as dag:
  def greeting():
    """Just check that the DAG is started in the log."""
    import logging
    logging.info('Bonjour le monde depuis le DAG 1')

  hello_python = python_operator.PythonOperator(
    task_id='bonjour',
    python_callable=greeting)

  goodbye_dummy = DummyOperator(task_id='au_revoir')

  hello_python >> goodbye_dummy

Ce code crée un DAG simple dans Airflow appelé "dag_1", qui contient deux tâches :

* La première tâche s'appelle "hello_python" et utilise l'opérateur Python pour exécuter la fonction "greeting()". Cette fonction écrit simplement "Bonjour le monde depuis le DAG 1" dans les logs.
* La deuxième tâche s'appelle "goodbye_dummy" et utilise l'opérateur DummyOperator pour simuler une tâche de fin. La flèche ">>" entre ces deux tâches définit une dépendance, de sorte que "goodbye_dummy" ne peut être exécuté que lorsque "hello_python" est terminé avec succès.
La ligne "with models.DAG(...)" crée un objet DAG avec les paramètres suivants :

* "dag_id": le nom du DAG (dans ce cas "dag_1").
* "schedule_interval": la fréquence à laquelle le DAG sera exécuté (dans ce cas toutes les minutes).
* "start_date": la date de début du DAG (dans ce cas, il commence immédiatement).
* "catchup": un booléen qui indique si Airflow doit rattraper les exécutions manquées depuis la date de début (dans ce cas, c'est désactivé).

En résumé, ce DAG exécutera la fonction "greeting()" toutes les minutes, écrira un message de log et terminera la tâche avec un opérateur "Dummy".

![image](https://user-images.githubusercontent.com/123757632/227970640-39e0893d-6b28-4aa8-ad65-9f09ef4f31f7.png)


Dans L'interface Utilisateur, Plus Précisément Dans L'interface Graphique Du Dag Nommé Dag_1, Toutes Les Tâches Peuvent Être Repérées.


![image](https://user-images.githubusercontent.com/123757632/227970692-7099a4d3-7d48-49a9-936e-cb9fe455e288.png)


### 2 - parent_dag_2.py

In [ ]:
from airflow import models
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators import python_operator
from airflow.utils.dates import days_ago


with models.DAG(
    'dag_2',
    schedule_interval='*/1 * * * *', # Chaque minute
    start_date=days_ago(0),
    catchup=False) as dag:
  def greeting():
    """Just check that the DAG is started in the log."""
    import logging
    logging.info('Bonjour le monde depuis le DAG 2')

  hello_python = python_operator.PythonOperator(
    task_id='bonjour',
    python_callable=greeting)

  goodbye_dummy = DummyOperator(task_id='au_revoir')

  hello_python >> goodbye_dummy

Ce code crée un DAG nommé "dag_2" dans Airflow qui est identique au DAG précédent ("dag_1") que nous avons vu. Il contient deux tâches :

* La première tâche s'appelle "hello_python" et utilise l'opérateur Python pour exécuter la fonction "greeting()". Cette fonction écrit simplement "Bonjour le monde depuis le DAG 2" dans les logs.
* La deuxième tâche s'appelle "goodbye_dummy" et utilise l'opérateur DummyOperator pour simuler une tâche de fin. La flèche ">>" entre ces deux tâches définit une dépendance, de sorte que "goodbye_dummy" ne peut être exécuté que lorsque "hello_python" est terminé avec succès.

La ligne "with models.DAG(...)" crée un objet DAG avec les paramètres suivants :

* "dag_id": le nom du DAG (dans ce cas "dag_2").
* "schedule_interval": la fréquence à laquelle le DAG sera exécuté (dans ce cas toutes les minutes).
* "start_date": la date de début du DAG (dans ce cas, il commence immédiatement).
* "catchup": un booléen qui indique si Airflow doit rattraper les exécutions manquées depuis la date de début (dans ce cas, c'est désactivé).

En résumé, ce DAG exécutera la fonction "greeting()" toutes les minutes, écrira un message de log et terminera la tâche avec un opérateur "Dummy". La seule différence entre ce DAG et le précédent est le nom.




![image](https://user-images.githubusercontent.com/123757632/227970722-d5a85af4-25eb-42e7-8526-e046587ee17b.png)



![image](https://user-images.githubusercontent.com/123757632/227970795-4eafaabe-c71b-4c1a-825a-c2e5be5201ab.png)


## 2 . Création du dag enfant 'dag en aval'

### child_master_dag.py

In [ ]:
"""Déclenchez les DAGs #1 et #2 et faites quelque chose s'ils réussissent."""

from airflow import DAG
from airflow.sensors.external_task import ExternalTaskSensor
from airflow.operators.dummy_operator import DummyOperator
from airflow.utils.dates import days_ago


with DAG(
    'Dag_master',
    schedule_interval='*/1 * * * *', # Chaque minute
    start_date=days_ago(0),
    catchup=False) as dag:
  def greeting():
    """Vérifiez simplement que le DAG a démarré dans le journal."""
    import logging
    logging.info('Hello World from DAG MASTER')

  externalsensor1 = ExternalTaskSensor(
    task_id='dag_1_completed_status',
    external_dag_id='dag_1',
    external_task_id=None, #Attendez que tout le DAG soit terminé
    check_existence=True,
    timeout=120)

  externalsensor2 = ExternalTaskSensor(
    task_id='dag_2_completed_status',
    external_dag_id='dag_2',
    external_task_id=None, # Attendez que tout le DAG soit terminé
    check_existence=True,
    timeout=120)

  goodbye_dummy = DummyOperator(task_id='goodbye_master')

  [externalsensor1, externalsensor2] >> goodbye_dummy

Le DAG principal est créé en utilisant la méthode DAG() de la bibliothèque Airflow. Le DAG est nommé "Dag_master" et est configuré pour s'exécuter toutes les minutes en utilisant le paramètre schedule_interval.

La date de début est définie comme étant la date d'aujourd'hui en utilisant la fonction days_ago(). Une fonction appelée greeting() est définie pour vérifier que le DAG a démarré en écrivant un message de journalisation ("Hello World from DAG MASTER").

Deux capteurs de tâches externes sont créés pour surveiller les deux DAGs #1 et #2 en attente de leur achèvement. 

Les capteurs sont configurés avec les paramètres suivants :

* task_id: l'identifiant de la tâche
* external_dag_id: l'identifiant du DAG externe à surveiller
* external_task_id: l'identifiant de la tâche externe à surveiller. Dans ce cas, nous ne surveillons pas une tâche spécifique, mais tout le DAG. Par conséquent, nous avons défini cette valeur sur None.
* check_existence: une valeur booléenne pour indiquer si le capteur doit vérifier l'existence de la tâche ou attendre qu'elle soit terminée.
* timeout: le temps maximum en secondes pendant lequel le capteur doit attendre avant de déclencher une exception.
* Finalement, une tâche "goodbye_master" est créée en utilisant l'opérateur fictif DummyOperator. Cette tâche sera exécutée si les deux capteurs de tâches externes réussissent. 
* Les deux capteurs sont connectés à la tâche "goodbye_master" en utilisant l'opérateur de décalage de bits (>>) pour indiquer que la tâche "goodbye_master" ne doit pas être exécutée tant que les deux capteurs ne se sont pas déclenchés.

En résumé, ce DAG crée deux capteurs de tâches externes pour surveiller deux autres DAGs et attend leur achèvement avant d'exécuter une tâche fictive. Ce code peut être modifié pour effectuer une tâche réelle une fois que les deux DAGs sont terminés avec succès.


![image](https://user-images.githubusercontent.com/123757632/227970827-38425e7e-132b-4219-acdd-9ab90f8967a5.png)


![image](https://user-images.githubusercontent.com/123757632/227970858-cb27f9ba-0f12-47c0-8cf6-a199da0f37ba.png)


## ACTIVATION DES DAGS

![image](https://user-images.githubusercontent.com/123757632/227970898-5c7dab64-9825-4f95-ab97-ea1f3d1ed219.png)


### ACTIVATION DES DAGS-DAG1


![image](https://user-images.githubusercontent.com/123757632/227970915-ae6b4578-7ecc-41c6-9344-b30781742a45.png)


### ACTIVATION DES DAGS-DAG2


![image](https://user-images.githubusercontent.com/123757632/227970954-4c1c4422-2ad2-454f-84c3-458caba15e8d.png)


### ACTIVATION DES DAGS-DAG_MASTER



![image](https://user-images.githubusercontent.com/123757632/227971011-b6b655df-99a5-4b6e-9d85-d673e236d52a.png)